# OpenAI Agents SDK - 처리 패턴

이 튜토리얼은 OpenAI Agents SDK에서 멀티 에이전트 워크플로우의 다양한 처리 패턴을 다룬다. 순차 처리, 병렬 처리, 조건부 라우팅 등의 핵심 패턴을 학습하여 효율적인 에이전트 시스템을 구축한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv pydantic

---

## 1. 순차 처리 (Sequential Processing)

이 챕터에서는 단계별 에이전트 워크플로우를 구축하는 방법을 다룬다.

### 학습 내용

- 각 에이전트가 이전 에이전트의 출력을 처리하는 선형 워크플로우 구축
- 워크플로우 단계 간 공유 컨텍스트와 중간 결과 유지
- 콘텐츠 파이프라인 생성 (리서치 → 분석 → 작성 → 편집)
- 워크플로우 진행 상황 및 단계 완료 시간 추적

### 환경 설정

In [1]:
import os
import asyncio
import time
from dotenv import load_dotenv
from pydantic import BaseModel, ConfigDict
from typing import Dict, Any
from datetime import datetime
from agents import Agent, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 워크플로우 컨텍스트 모델

In [2]:
class WorkflowContext(BaseModel):
    """
    순차 워크플로우의 컨텍스트를 관리하는 모델이다.
    각 단계의 결과와 메타데이터를 추적한다.
    """
    model_config = ConfigDict(strict=True)
    
    workflow_id: str                          # 워크플로우 ID
    topic: str                                # 처리 주제
    stage_results: Dict[str, str] = {}        # 단계별 결과
    metadata: Dict[str, Any] = {}             # 메타데이터
    created_at: datetime                      # 생성 시간
    
    def add_stage_result(self, agent_name: str, result: str):
        """워크플로우 단계 결과를 추가한다."""
        self.stage_results[agent_name] = result
        self.metadata[f"{agent_name}_completed_at"] = datetime.now().isoformat()

print("WorkflowContext 모델이 정의되었다.")

WorkflowContext 모델이 정의되었다.


### 순차 워크플로우 에이전트 생성

콘텐츠 생성 파이프라인을 위한 전문화된 에이전트를 생성한다.

In [3]:
# 리서치 에이전트
research_agent = Agent(
    name="Research Agent",
    instructions="""
    주어진 주제에 대해 철저히 조사하세요.
    핵심 사실, 통계, 인사이트를 제공하세요.
    """
)

# 분석 에이전트
analysis_agent = Agent(
    name="Analysis Agent", 
    instructions="""
    리서치 데이터를 분석하세요.
    패턴, 트렌드, 핵심 시사점을 식별하세요.
    """
)

# 작성 에이전트
writer_agent = Agent(
    name="Writer Agent",
    instructions="""
    제공된 리서치와 분석을 바탕으로 포괄적인 글을 작성하세요.
    명확하고 구조화된 콘텐츠를 생성하세요.
    """
)

# 편집 에이전트
editor_agent = Agent(
    name="Editor Agent",
    instructions="""
    작성된 콘텐츠를 편집하고 다듬으세요.
    명확성, 흐름, 가독성을 개선하세요.
    """
)

print("순차 워크플로우 에이전트가 생성되었다:")
print(f"  1. {research_agent.name}")
print(f"  2. {analysis_agent.name}")
print(f"  3. {writer_agent.name}")
print(f"  4. {editor_agent.name}")

순차 워크플로우 에이전트가 생성되었다:
  1. Research Agent
  2. Analysis Agent
  3. Writer Agent
  4. Editor Agent


### 순차 처리 워크플로우 함수

In [4]:
async def sequential_workflow(initial_input: str, context: WorkflowContext) -> str:
    """
    순차 처리 워크플로우 - 각 에이전트가 이전 에이전트 완료를 기다린다.
    
    Args:
        initial_input: 초기 입력
        context: 워크플로우 컨텍스트
    
    Returns:
        최종 처리 결과
    """
    
    agents = [research_agent, analysis_agent, writer_agent, editor_agent]
    current_input = initial_input
    
    print(f"🔄 순차 워크플로우: {context.workflow_id}")
    print(f"주제: {context.topic}")
    print("=" * 60)
    
    for i, agent in enumerate(agents, 1):
        print(f"\n{i}️⃣ {agent.name}로 처리 중...")
        start_time = time.time()
        
        # 컨텍스트 인식 프롬프트 구성
        if i == 1:
            # 첫 번째 에이전트는 원본 입력 받음
            prompt = f"이 주제를 조사하세요: {current_input}"
        else:
            # 이후 에이전트는 이전 결과 + 컨텍스트 받음
            previous_results = "\n".join([
                f"- {name}: {result[:100]}..." 
                for name, result in context.stage_results.items()
            ])
            prompt = f"""
            주제 '{context.topic}'에 대한 워크플로우를 계속하세요.
            
            이전 단계 결과:
            {previous_results}
            
            처리할 현재 입력: {current_input}
            """
        
        # 에이전트 실행 (블로킹 - 완료 대기)
        result = await Runner.run(agent, prompt)
        
        elapsed_time = time.time() - start_time
        
        # 이 에이전트의 출력을 다음 에이전트의 입력으로 사용
        current_input = result.final_output
        
        # 중간 결과로 공유 컨텍스트 업데이트
        context.add_stage_result(agent.name, result.final_output)
        
        print(f"✅ {agent.name} 완료 ({elapsed_time:.2f}초)")
        print(f"출력 미리보기: {current_input[:150]}...")
        print("-" * 40)
    
    print(f"\n🎉 순차 워크플로우 완료")
    return current_input

print("sequential_workflow 함수가 정의되었다.")

sequential_workflow 함수가 정의되었다.


### 순차 처리 데모

In [5]:
async def demo_sequential():
    """
    순차 처리를 시연한다.
    """
    
    topics = [
        "금융 분야의 인공지능 활용",
    ]
    
    for topic in topics:
        print(f"\n{'='*80}")
        print(f"순차 처리 데모: {topic}")
        print("="*80)
        
        # 워크플로우 컨텍스트 생성
        context = WorkflowContext(
            workflow_id=f"SEQ-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
            topic=topic,
            created_at=datetime.now()
        )
        
        # 순차 워크플로우 실행
        final_result = await sequential_workflow(topic, context)
        
        # 워크플로우 요약 표시
        print(f"\n📋 워크플로우 요약:")
        print(f"워크플로우 ID: {context.workflow_id}")
        print(f"총 단계 수: {len(context.stage_results)}")
        print(f"처리 순서:")
        for i, (stage, _) in enumerate(context.stage_results.items(), 1):
            completion_time = context.metadata.get(f"{stage}_completed_at", "Unknown")
            print(f"  {i}. {stage} - 완료: {completion_time}")
        
        print(f"\n🎉 최종 결과:")
        print(f"{final_result[:300]}...")

# 데모 실행
await demo_sequential()


순차 처리 데모: 금융 분야의 인공지능 활용
🔄 순차 워크플로우: SEQ-20251213-111925
주제: 금융 분야의 인공지능 활용

1️⃣ Research Agent로 처리 중...
✅ Research Agent 완료 (16.33초)
출력 미리보기: 주제: 금융 분야의 인공지능(AI) 활용

1. 개요  
인공지능(AI)은 방대한 데이터를 분석하여 패턴을 찾아내고, 예측과 자동화를 실현할 수 있는 기술로, 금융 분야에서 혁신을 촉진하는 핵심 역할을 합니다. 최근 빅데이터 및 컴퓨팅 기술의 발전과 함께 금융기관은 고...
----------------------------------------

2️⃣ Analysis Agent로 처리 중...
✅ Analysis Agent 완료 (11.11초)
출력 미리보기: 분석 결과 요약  
아래는 입력하신 리서치 데이터에 대한 패턴, 트렌드, 핵심 시사점 분석입니다.

---

### 1. **패턴(패턴 식별)**
- **적용 확산**: 금융 분야 전체(은행, 증권, 보험 등)에서 AI 기술 도입이 빠르게 확산되고 있음
- **주요 현...
----------------------------------------

3️⃣ Writer Agent로 처리 중...
✅ Writer Agent 완료 (37.68초)
출력 미리보기: 금융 분야의 인공지능(AI) 활용  
— 패턴, 트렌드, 핵심 시사점 분석을 바탕으로

---

## 1. 개요  
인공지능(AI)은 방대한 금융 데이터를 빠르게 처리·분석하고, 정교한 패턴과 규칙을 도출해 미래를 예측하고 업무를 자동화하는 기술입니다. 최근 금융 산업...
----------------------------------------

4️⃣ Editor Agent로 처리 중...
✅ Editor Agent 완료 (8.60초)
출력 미리보기: 아래는 주어진 콘텐츠의 명확성, 흐름, 가독성을 개선한 편집본입니다.

---

## 1. 개요  
인공지능(AI)은 방대한 금융 데이터를 신

---

## 2. 병렬 처리 (Parallel Processing)

이 챕터에서는 동시 멀티 에이전트 실행을 다룬다.

### 학습 내용

- asyncio.gather()를 사용한 여러 에이전트 동시 실행
- 순차 실행과 병렬 실행 간 성능 비교
- 적절한 에러 처리를 통한 병렬 에이전트 워크플로우 관리
- 속도와 효율성을 위한 멀티 에이전트 시스템 최적화

### 병렬 컨텍스트 모델

In [6]:
class ParallelContext(BaseModel):
    """
    병렬 워크플로우의 컨텍스트를 관리하는 모델이다.
    병렬 실행 결과와 실행 시간을 추적한다.
    """
    model_config = ConfigDict(strict=True)
    
    workflow_id: str                              # 워크플로우 ID
    topic: str                                    # 처리 주제
    parallel_results: Dict[str, str] = {}         # 병렬 실행 결과
    execution_times: Dict[str, float] = {}        # 실행 시간
    created_at: datetime                          # 생성 시간
    
    def add_result(self, agent_name: str, result: str, execution_time: float):
        """병렬 실행 결과를 추가한다."""
        self.parallel_results[agent_name] = result
        self.execution_times[agent_name] = execution_time

print("ParallelContext 모델이 정의되었다.")

ParallelContext 모델이 정의되었다.


### 병렬 처리 워크플로우 함수

In [7]:
async def parallel_workflow(topic: str, context: ParallelContext) -> Dict[str, str]:
    """
    병렬 처리 - 에이전트들이 동시에 작업한다.
    
    Args:
        topic: 처리 주제
        context: 병렬 컨텍스트
    
    Returns:
        병렬 처리 결과 딕셔너리
    """
    
    print(f"⚡ 병렬 워크플로우: {context.workflow_id}")
    print(f"주제: {topic}")
    print("=" * 60)
    
    # 병렬 실행을 위한 전문 리서치 에이전트 생성
    market_researcher = Agent(
        name="Market Researcher",
        instructions="주제의 시장 트렌드, 통계, 비즈니스 측면을 조사하세요."
    )
    
    tech_researcher = Agent(
        name="Tech Researcher", 
        instructions="주제와 관련된 기술적 측면, 혁신, 개발 동향을 조사하세요."
    )
    
    social_researcher = Agent(
        name="Social Researcher",
        instructions="주제의 사회적 영향, 사용자 행동, 문화적 측면을 조사하세요."
    )
    
    competitive_researcher = Agent(
        name="Competitive Researcher",
        instructions="경쟁자, 시장 포지셔닝, 경쟁 환경을 조사하세요."
    )
    
    # 모든 에이전트를 병렬로 실행
    print("🚀 병렬 리서치 작업 시작...")
    start_time = time.time()
    
    # 동시 실행을 위한 태스크 생성
    tasks = [
        Runner.run(market_researcher, f"다음 주제의 시장 측면을 조사하세요: {topic}"),
        Runner.run(tech_researcher, f"다음 주제의 기술 측면을 조사하세요: {topic}"),
        Runner.run(social_researcher, f"다음 주제의 사회적 측면을 조사하세요: {topic}"),
        Runner.run(competitive_researcher, f"다음 주제의 경쟁 환경을 조사하세요: {topic}")
    ]
    
    # 모든 태스크가 동시에 완료될 때까지 대기
    results = await asyncio.gather(*tasks)
    
    total_time = time.time() - start_time
    
    # 타이밍과 함께 결과 수집
    agent_names = ["Market Researcher", "Tech Researcher", "Social Researcher", "Competitive Researcher"]
    
    for agent_name, result in zip(agent_names, results):
        context.add_result(agent_name, result.final_output, total_time / len(agent_names))
        print(f"✅ {agent_name} 완료")
    
    print(f"⚡ 모든 병렬 태스크 완료: {total_time:.2f}초")
    return context.parallel_results

print("parallel_workflow 함수가 정의되었다.")

parallel_workflow 함수가 정의되었다.


### 순차 vs 병렬 성능 비교

In [8]:
async def compare_sequential_vs_parallel(topic: str):
    """
    순차 실행과 병렬 실행 시간을 비교한다.
    
    Args:
        topic: 비교에 사용할 주제
    """
    
    print(f"\n⚖️ 성능 비교: {topic}")
    print("=" * 60)
    
    # 비교용 에이전트 생성
    agents = [
        Agent(name="Agent 1", instructions="시장 측면을 조사하세요"),
        Agent(name="Agent 2", instructions="기술 측면을 조사하세요"), 
        Agent(name="Agent 3", instructions="사회적 측면을 조사하세요"),
        Agent(name="Agent 4", instructions="경쟁 측면을 조사하세요")
    ]
    
    # 순차 실행 (하나씩 차례대로)
    print("🐌 순차 실행:")
    seq_start = time.time()
    
    for agent in agents:
        result = await Runner.run(agent, f"{topic} 조사")
        print(f"  ✅ {agent.name} 완료")
    
    seq_time = time.time() - seq_start
    print(f"순차 실행 총 시간: {seq_time:.2f}초")
    
    # 병렬 실행 (동시에)
    print("\n⚡ 병렬 실행:")
    par_start = time.time()
    
    tasks = [Runner.run(agent, f"{topic} 조사") for agent in agents]
    await asyncio.gather(*tasks)
    
    for agent in agents:
        print(f"  ✅ {agent.name} 완료")
    
    par_time = time.time() - par_start
    print(f"병렬 실행 총 시간: {par_time:.2f}초")
    
    # 성능 개선 표시
    speedup = seq_time / par_time if par_time > 0 else 0
    print(f"\n📊 성능 개선: {speedup:.1f}배 빠름")
    print(f"절약된 시간: {seq_time - par_time:.2f}초")

print("compare_sequential_vs_parallel 함수가 정의되었다.")

compare_sequential_vs_parallel 함수가 정의되었다.


### 병렬 처리 데모

In [9]:
async def demo_parallel():
    """
    병렬 처리를 시연한다.
    """
    
    topic = "블록체인 기술"
    
    print(f"\n{'='*80}")
    print(f"병렬 처리 데모: {topic}")
    print("="*80)
    
    # 병렬 컨텍스트 생성
    context = ParallelContext(
        workflow_id=f"PAR-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        topic=topic,
        created_at=datetime.now()
    )
    
    # 병렬 워크플로우 실행
    results = await parallel_workflow(topic, context)
    
    # 결과 요약 표시
    print(f"\n📋 병렬 실행 요약:")
    print(f"워크플로우 ID: {context.workflow_id}")
    print(f"동시 태스크 수: {len(results)}")
    print(f"결과:")
    for agent_name, result in results.items():
        execution_time = context.execution_times.get(agent_name, 0)
        print(f"  • {agent_name}: {result[:80]}... (약 {execution_time:.2f}초)")
    
    # 성능 비교
    await compare_sequential_vs_parallel(topic)

# 데모 실행
await demo_parallel()


병렬 처리 데모: 블록체인 기술
⚡ 병렬 워크플로우: PAR-20251213-112039
주제: 블록체인 기술
🚀 병렬 리서치 작업 시작...
✅ Market Researcher 완료
✅ Tech Researcher 완료
✅ Social Researcher 완료
✅ Competitive Researcher 완료
⚡ 모든 병렬 태스크 완료: 19.10초

📋 병렬 실행 요약:
워크플로우 ID: PAR-20251213-112039
동시 태스크 수: 4
결과:
  • Market Researcher: 블록체인 기술 시장 조사 (2024년 기준)

**1. 시장 트렌드**

- **기업 활용 확대**: 금융(FinTech)을 넘어 공급망 관리,... (약 4.78초)
  • Tech Researcher: 네, 아래에 **블록체인 기술**의 기술적 측면, 혁신, 최근 개발 동향을 정리했습니다.

---

### 1. 기술적 측면

#### 1) 분... (약 4.78초)
  • Social Researcher: 네, 블록체인 기술의 사회적 측면을 아래와 같이 정리할 수 있습니다.

---

### 1. 사회적 영향

#### **투명성의 증대**
- *... (약 4.78초)
  • Competitive Researcher: 네, “블록체인 기술”의 경쟁 환경에 대해 조사하여 아래와 같이 정리합니다.

---

## 1. 블록체인 시장 개요

- **정의:** 분산원... (약 4.78초)

⚖️ 성능 비교: 블록체인 기술
🐌 순차 실행:
  ✅ Agent 1 완료
  ✅ Agent 2 완료
  ✅ Agent 3 완료
  ✅ Agent 4 완료
순차 실행 총 시간: 51.93초

⚡ 병렬 실행:
  ✅ Agent 1 완료
  ✅ Agent 2 완료
  ✅ Agent 3 완료
  ✅ Agent 4 완료
병렬 실행 총 시간: 26.93초

📊 성능 개선: 1.9배 빠름
절약된 시간: 25.00초


---

## 조건부 라우팅 (Conditional Routing)

이 챕터에서는 처리 결과에 따른 동적 워크플로우 라우팅을 다룬다.

### 학습 내용

- 에이전트 응답을 사용하여 다음 워크플로우 단계 결정
- 여러 라우팅 경로가 있는 결정 트리 구축
- 다양한 신뢰도 수준과 에러 조건 처리
- 처리 결과에 반응하는 적응형 워크플로우 생성

### 처리 컨텍스트 클래스

In [10]:
class ProcessingContext:
    """
    조건부 라우팅 워크플로우의 컨텍스트를 관리하는 클래스이다.
    라우팅 기록을 추적한다.
    """
    
    def __init__(self, request_id: str, user_type: str, priority: str):
        self.request_id = request_id          # 요청 ID
        self.user_type = user_type            # 사용자 유형
        self.priority = priority              # 우선순위
        self.routing_history = []             # 라우팅 기록
        self.created_at = datetime.now()      # 생성 시간

print("ProcessingContext 클래스가 정의되었다.")

ProcessingContext 클래스가 정의되었다.


### 조건부 라우팅 에이전트 생성

In [11]:
# 처리기 에이전트 - 라우팅 결정을 내림
processor_agent = Agent(
    name="Processor Agent",
    instructions="""
    입력을 분석하고 라우팅 결정을 제공하세요.
    
    요청에 따라 다음 결과 중 하나로 응답하세요:
    - "SUCCESS_HIGH": 높은 신뢰도로 처리할 수 있는 간단한 요청
    - "SUCCESS_MEDIUM": 약간의 불확실성이 있지만 처리할 수 있는 요청
    - "NEEDS_REVIEW": 사람 검토가 필요한 모호한 요청
    - "REQUIRES_ESCALATION": 복잡하거나 중요한 문제
    - "ERROR": 잘못되었거나 유효하지 않은 요청
    
    응답을 결과로 시작하고 이유를 설명하세요.
    """
)

# 최종 처리 에이전트
finalizer_agent = Agent(
    name="Finalizer Agent",
    instructions="성공적인 처리 결과를 최종화하세요. 다듬어진 최종 출력을 제공하세요."
)

# 검토 에이전트
reviewer_agent = Agent(
    name="Reviewer Agent", 
    instructions="불확실한 결과를 검토하세요. 품질 평가와 개선 사항을 제공하세요."
)

# 사람 검토 에이전트
human_review_agent = Agent(
    name="Human Review Agent",
    instructions="사람 검토가 필요한 케이스를 처리하세요. 사람의 감독을 위한 상세 분석을 제공하세요."
)

# 관리자 에이전트
supervisor_agent = Agent(
    name="Supervisor Agent",
    instructions="에스컬레이션된 케이스를 처리하세요. 관리 수준의 해결책을 제공하세요."
)

# 에러 처리 에이전트
error_handler_agent = Agent(
    name="Error Handler Agent",
    instructions="처리 에러를 처리하세요. 에러 분석과 복구 옵션을 제공하세요."
)

print("조건부 라우팅 에이전트가 생성되었다:")
print(f"  - {processor_agent.name} (초기 분류)")
print(f"  - {finalizer_agent.name} (성공 처리)")
print(f"  - {reviewer_agent.name} (중간 신뢰도)")
print(f"  - {human_review_agent.name} (사람 검토)")
print(f"  - {supervisor_agent.name} (에스컬레이션)")
print(f"  - {error_handler_agent.name} (에러 처리)")

조건부 라우팅 에이전트가 생성되었다:
  - Processor Agent (초기 분류)
  - Finalizer Agent (성공 처리)
  - Reviewer Agent (중간 신뢰도)
  - Human Review Agent (사람 검토)
  - Supervisor Agent (에스컬레이션)
  - Error Handler Agent (에러 처리)


### 조건부 워크플로우 함수

In [12]:
async def conditional_workflow(input_data: str, context: ProcessingContext) -> str:
    """
    처리 결과에 따른 조건부 라우팅 워크플로우이다.
    
    Args:
        input_data: 입력 데이터
        context: 처리 컨텍스트
    
    Returns:
        최종 처리 결과
    """
    
    print(f"🔀 조건부 라우팅 워크플로우")
    print(f"요청 ID: {context.request_id}")
    print(f"입력: {input_data}")
    print("=" * 60)
    
    # 단계 1: 초기 처리
    print("1️⃣ 초기 처리...")
    context.routing_history.append("Processor Agent로 시작")
    
    result = await Runner.run(processor_agent, f"이 요청을 처리하세요: {input_data}")
    processor_response = result.final_output
    
    print(f"처리기 응답: {processor_response[:100]}...")
    
    # 응답에서 결과 추출
    outcome = "SUCCESS_MEDIUM"  # 기본값
    if "SUCCESS_HIGH" in processor_response:
        outcome = "SUCCESS_HIGH"
    elif "SUCCESS_MEDIUM" in processor_response:
        outcome = "SUCCESS_MEDIUM"
    elif "NEEDS_REVIEW" in processor_response:
        outcome = "NEEDS_REVIEW"
    elif "REQUIRES_ESCALATION" in processor_response:
        outcome = "REQUIRES_ESCALATION"
    elif "ERROR" in processor_response:
        outcome = "ERROR"
    
    # 단계 2: 결과에 따라 라우팅
    print(f"\n2️⃣ 조건부 라우팅: {outcome}")
    
    if outcome == "SUCCESS_HIGH":
        # 높은 신뢰도 - 직접 최종화로
        print("🎯 높은 신뢰도 → Finalizer로 직행")
        context.routing_history.append("Finalizer로 라우팅 (높은 신뢰도)")
        final_result = await Runner.run(finalizer_agent, f"이 요청을 최종화하세요: {input_data}")
        
    elif outcome == "SUCCESS_MEDIUM":
        # 중간 신뢰도 - 먼저 검토 받기
        print("🔍 중간 신뢰도 → 검토 후 최종화")
        context.routing_history.append("Reviewer로 라우팅 (중간 신뢰도)")
        review_result = await Runner.run(reviewer_agent, f"이 요청을 검토하세요: {input_data}")
        
        context.routing_history.append("검토 완료, 최종화 진행")
        final_result = await Runner.run(finalizer_agent, f"검토된 콘텐츠를 최종화하세요: {review_result.final_output}")
    
    elif outcome == "NEEDS_REVIEW":
        # 사람 검토 에이전트로 전송
        print("👤 검토 필요 → Human Review Agent")
        context.routing_history.append("Human Review Agent로 라우팅")
        final_result = await Runner.run(human_review_agent, f"사람 검토 필요: {input_data}")
    
    elif outcome == "REQUIRES_ESCALATION":
        # 관리자에게 에스컬레이션
        print("⬆️ 에스컬레이션 → Supervisor Agent")
        context.routing_history.append("Supervisor Agent로 에스컬레이션")
        final_result = await Runner.run(supervisor_agent, f"에스컬레이션된 케이스: {input_data}")
    
    else:  # ERROR
        # 에러 처리 에이전트로 전송
        print("❌ 에러 → Error Handler Agent")
        context.routing_history.append("에러 처리 트리거됨")
        final_result = await Runner.run(error_handler_agent, f"요청의 에러를 처리하세요: {input_data}")
    
    return final_result.final_output

print("conditional_workflow 함수가 정의되었다.")

conditional_workflow 함수가 정의되었다.


### 조건부 라우팅 데모

In [13]:
async def demo_conditional_routing():
    """
    다양한 시나리오로 조건부 라우팅을 시연한다.
    """
    
    test_scenarios = [
        {
            "input": "비밀번호 재설정 도움이 필요해요",
            "context": ProcessingContext("REQ-001", "standard", "normal"),
            "description": "간단한 요청 - 높은 신뢰도로 성공 예상"
        },
        {
            "input": "여러 API와 관련된 복잡한 통합 문제가 있어요",
            "context": ProcessingContext("REQ-002", "enterprise", "high"),
            "description": "복잡한 요청 - 검토 또는 에스컬레이션 필요 가능"
        },
        {
            "input": "시스템이 다운되고 모든 데이터를 잃었어요, 긴급해요!",
            "context": ProcessingContext("REQ-003", "premium", "critical"),
            "description": "중요 문제 - 에스컬레이션 필요 가능"
        },
        {
            "input": "잘못된 데이터가 있는 유효하지 않은 요청 @#$%^&*()",
            "context": ProcessingContext("REQ-004", "unknown", "low"),
            "description": "잘못된 요청 - 에러 처리 트리거 예상"
        }
    ]
    
    for scenario in test_scenarios:
        print(f"\n{'='*80}")
        print(f"시나리오: {scenario['description']}")
        print("="*80)
        
        final_response = await conditional_workflow(scenario["input"], scenario["context"])
        
        print(f"\n📋 라우팅 요약:")
        print(f"요청 ID: {scenario['context'].request_id}")
        print(f"라우팅 경로:")
        for i, step in enumerate(scenario["context"].routing_history, 1):
            print(f"  {i}. {step}")
        
        print(f"\n🎯 최종 응답:")
        print(f"{final_response[:200]}..." if len(final_response) > 200 else final_response)
        print("-" * 80)

# 데모 실행
await demo_conditional_routing()


시나리오: 간단한 요청 - 높은 신뢰도로 성공 예상
🔀 조건부 라우팅 워크플로우
요청 ID: REQ-001
입력: 비밀번호 재설정 도움이 필요해요
1️⃣ 초기 처리...
처리기 응답: SUCCESS_HIGH: 비밀번호 재설정 요청은 일반적으로 표준 절차가 명확히 정해져 있고, 자동화된 시스템 또는 안내문을 통해 지원할 수 있으므로 높은 신뢰도로 처리할 수 있습니...

2️⃣ 조건부 라우팅: SUCCESS_HIGH
🎯 높은 신뢰도 → Finalizer로 직행

📋 라우팅 요약:
요청 ID: REQ-001
라우팅 경로:
  1. Processor Agent로 시작
  2. Finalizer로 라우팅 (높은 신뢰도)

🎯 최종 응답:
비밀번호 재설정 안내

비밀번호 재설정을 도와드리겠습니다. 아래 절차를 따라주세요:

1. 로그인 화면에서 "비밀번호를 잊으셨나요?" 또는 "비밀번호 재설정" 링크를 클릭하세요.
2. 계정에 등록된 이메일 주소나 사용자명을 입력하세요.
3. 입력하신 이메일 주소로 전송된 비밀번호 재설정 메일을 확인하세요.
4. 이메일 내의 안내에 따라 새로운 비밀번호를 설...
--------------------------------------------------------------------------------

시나리오: 복잡한 요청 - 검토 또는 에스컬레이션 필요 가능
🔀 조건부 라우팅 워크플로우
요청 ID: REQ-002
입력: 여러 API와 관련된 복잡한 통합 문제가 있어요
1️⃣ 초기 처리...
처리기 응답: REQUIRES_ESCALATION: 요청에서 여러 API와 관련된 복잡한 통합 문제가 언급되어 있습니다. 이는 복잡하거나 중요한 문제로 분류되며, 자동화된 처리가 아닌 전문가의 ...

2️⃣ 조건부 라우팅: REQUIRES_ESCALATION
⬆️ 에스컬레이션 → Supervisor Agent

📋 라우팅 요약:
요청 ID: REQ-002
라우팅 경로:
  1. Processor Agent로 시작
  2. Supervis

---

## 처리 패턴 요약

1. 순차 처리 (Sequential Processing)
   - 각 에이전트가 이전 에이전트 완료 후 실행
   - 결과가 다음 단계의 입력이 됨
   - 사용 사례: 콘텐츠 파이프라인, 문서 처리
   - 장점: 예측 가능, 디버깅 용이
   - 단점: 총 실행 시간이 김

2. 병렬 처리 (Parallel Processing)
   - asyncio.gather()로 여러 에이전트 동시 실행
   - 독립적인 작업에 적합
   - 사용 사례: 다중 리서치, 병렬 분석
   - 장점: 빠른 실행, 효율적
   - 단점: 에러 처리 복잡

3. 조건부 라우팅 (Conditional Routing)
   - 처리 결과에 따라 동적으로 다음 단계 결정
   - 여러 라우팅 경로 지원
   - 사용 사례: 고객 지원, 요청 분류
   - 장점: 유연성, 적응성
   - 단점: 복잡한 설계 필요

패턴 선택 가이드:
   - 단계별 의존성 있음 → 순차 처리
   - 독립적 작업 + 속도 필요 → 병렬 처리
   - 동적 의사결정 필요 → 조건부 라우팅
   - 복잡한 워크플로우 → 패턴 조합

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 처리 패턴을 다루었다. 다음과 같은 내용을 학습하였다:

1. **순차 처리**
   - WorkflowContext로 단계별 결과 추적
   - 콘텐츠 파이프라인 구축 (리서치 → 분석 → 작성 → 편집)
   - 각 에이전트 간 컨텍스트 전달

2. **병렬 처리**
   - ParallelContext로 동시 실행 결과 관리
   - asyncio.gather()를 사용한 동시 실행
   - 순차 vs 병렬 성능 비교

3. **조건부 라우팅**
   - ProcessingContext로 라우팅 기록 추적
   - 처리 결과에 따른 동적 경로 선택
   - 다양한 신뢰도 수준과 에러 조건 처리

이러한 패턴을 조합하면 복잡하고 효율적인 멀티 에이전트 워크플로우를 구축할 수 있다.